In [26]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
import seaborn as sns

In [27]:
pages = np.arange(1, 500, 50) 
headers = {'Accept-Language' : 'en-US,en; q=0.8'}

titles = []
years = []
ratings = []
genres = []
runtimes = []
imdb_ratings = []
imdb_ratings_standardized = []
metascores = []
votes = []

for page in pages:
    response = get("https://www.imdb.com/search/title?genres=sci-fi&"+"start"+str(page)+ "&explore=title_type, genres&ref = adv_prv", headers=headers)
    
    sleep(randint(8,15))
    
    if response.status_code != 200:
       warn('Request: {}; Status code: {}'.format(requests, response.status_code))
             
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    
    for container in movie_containers:
        title = container.h3.a.text
        titles.append(title)
        if container.h3.find('span', class_ = 'lister-item-year text-muted unbold') is not None:
            year = container.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
            years.append(year)
        else:
            years.append(None)
            
        if container.p.find('span', class_ = 'certificate') is not None:
            rating = container.p.find('span', class_ = 'certificate').text 
            ratings.append(rating)
        else:
            ratings.append("")
        
        if container.p.find('span', class_ = 'genre') is not None:
            genre  = container.p.find('span', class_ = 'genre').text.replace('\n', "").rstrip().split(',')
            genres.append(genre)
        else:
            genres.append("")
        
        if container.p.find('span', class_ = 'runtime') is not None:
            time = int(container.p.find('span', class_ = 'runtime').text.replace(" min", ""))
            runtimes.append(time)
        else:
            runtimes.append(None)
        '''
        if float(container.strong.text) is not None:
            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)
        else:
            imdb_ratings.append(None)
        '''
        if container.p.find('span', class_ = 'metascore') is not None:
            m_score = int(container.p.find('span', class_ = 'metascore').text)
            metascores.append(m_score)
        else:
            metascores.append(None)
        '''
        if container.find('span', attrs = {'name':'nv'})['data-value'] is not None:
            vote = int(container.find('span', attrs = {'name':'nv'})['data-value'])
            votes.append(vote)
        else:
            votes.append(None)
        '''

TypeError: 'NoneType' object is not subscriptable

In [ ]:
print(len(titles))
print(len(years))
print(len(ratings))
print(len(genres))
print(len(runtimes))
print(len(imdb_ratings))
print(len(metascores))
print(len(votes))







50
50
1
1
1
1
1
1


In [ ]:
sci_fi_df = pd.DataFrame({'movie': titles, 'year': years, 'rating': ratings, 'genre': genres,'runtime_min': runtimes, 'imdb': imdb_ratings, 'metascore': metascores, 'votes': votes})
sci_fi_df.loc[:, 'year'] = sci_fi_df['year'].str[-5:-1]
sci_fi_df['n_imdb'] = sci_fi_df['imdb'] * 10
final_df = sci_fi_df.loc[sci_fi_df['year'] != 'ovie']
final_df.loc[:, 'year'] = pd.to_numeric(final_df['year'])

ValueError: All arrays must be of the same length